In [11]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import os
import sys
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Bar
init_notebook_mode(connected=True)

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from analyser import Analyser
from db import SqliteDb as db

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Documentation

## Research Questions

** Which contribution have bots to the quality of Wikidata? **
* Which differences exist between bots which successfully completed the request for permission process and those who doesn't? (Hypothesis: The request for permission process is a quality gate for bots)
* How does bot's field of activity change over time?
* What is the difference between a bot with the group 'bot' and a bot with a botflag?

## Data preparation
### Request for permissions

As a first step we downloaded and parsed the data about requests for permissions about bots from Wikidata in May 2018. We collected the already closed requests which can be found in the Wikidata archive (e.g. https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Archive#Requests_for_bot_flags). The data is stored in a database.

Request for permissions which have no own page were left out (e.g. https://www.wikidata.org/w/index.php?title=Wikidata:Requests_for_permissions/Bot/Checkallthestrings_bot_1&action=edit&redlink=1).

Request for permissions which are listed several times in different archives were only parsed once (e.g. https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Bot/VIAFbot is listes in https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/RfBot/March_2013 and in https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/RfBot/April_2013).

## DB

### Schema

The db schema looks as follows (as diagram):

Note: The lates update for this diagram was on May 22 and may not be up-to-date!

![DB schema](images/db_schema.png)

and as ddl: 

```sql
CREATE TABLE requests_for_permissions (
    id                    INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    url                   TEXT UNIQUE,
    bot                   TEXT,
    bot_name              TEXT,
    bot_has_red_link      INTEGER,
    operator              INTEGER,
    is_successful         INTEGER,
    first_edit            DATE,
    last_edit             DATE,
    closed_at             DATE,
    revision_count        INTEGER,
    editor_count          INTEGER,
    comment_symbol_count  INTEGER,
    question_symbol_count INTEGER,
    oppose_symbol_count   INTEGER,
    answer_symbol_count   INTEGER,
    support_symbol_count  INTEGER,
    html                  TEXT,
    task                  TEXT,
    code                  TEXT,
    function              TEXT,
    archive_comment       TEXT,
    summary               TEXT,
    retrieved_at          DATE,
    FOREIGN KEY(bot) REFERENCES bot(id)
);
CREATE TABLE bots (
    id               INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    userid           INTEGER UNIQUE,
    name             TEXT UNIQUE,
    has_botflag      INTEGER,
    is_extension_bot INTEGER,
    is_blocked       INTEGER,
    blockid          INTEGER,
    blockedby        TEXT,
    blockedbyid      INTEGER,
    blockedtimestamp DATE,
    blockreason      TEXT,
    blockexpiry      TEXT,
    groups           TEXT,
    implicitgroups   TEXT,
    rights           TEXT,
    editcount        INTEGER,
    registration     TEXT,
    retrieved_at     DATE
);
```

### Tables

The tabel `requests_for_permissions` contains the following content:

| column                | content explanation                                                                   |
| ----------------------|:-------------------------------------------------------------------------------------:|
| id                    | unique identifier                                                                     |
| url                   | url of the request for permission                                                     |
| bot                   | the wikidata user account for which the request for permission is for.                |
| operator              | the wikidata user account which opend the request for permission and is responsible for the bot. |
| is_successful         | flag which states if the request for permission was successful. 1 if it was successful and 0 if not     |
| first_edit            | utc date and time when the first edit of this request for permission was made         |
| last_edit             | utc date and time when the last edit of this request for permission was made          |
| closed_at             | utc date and time when the request for permission was closed which is equvalent to the time when the summary was written |
| revision_count        | count how many revisions were applied to the request for permission                   |
| edit_count            | count how many distinct editors made revisons to the request for permission           |
| comment_symbol_count  | count how often a comment symbol was used in the discussion of the request            |
| question_symbol_count | count how often a question symbol was used in the discussion of the request           |
| oppose_symbol_count   | count how often a oppose symbol was used in the discussion of the request             |
| answer_symbol_count   | count how often a answer symbol was used in the discussion of the request             |
| support_symbol_count  | count how often a support symbol was used in the discussion of the request            |
| html                  | raw html of the site that can be found under the url. only the main part of the site is saved. |
| task                  | short description of the bot function                                                 |
| code                  | link to the online repository or other code describing text                           |
| function              | detailed description of the bot function                                              |
| archive_comment       | short comment why the request for permission was successful or not. Example can be found under the following link: https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/RfBot/April_2013 (see for example '(approved temporary)') |
| summary               | summary about the discussion and the final decision about the request for permission   | 
| retrieved_at          | utc date and time when the data was retrived                                          |

The tabel `bots` contains the following content:

| column                | content explanation                                                                   |
| ----------------------|:-------------------------------------------------------------------------------------:|
| name                  | name of the bot                                                                       |

## Analysis

### Analysis of Wikidata Bots

This chapter aims to analyse the bots on wikidata. 

As a first step we try to find out which user accounts on Wikidata are bots. There are several sources that contain lists of bots or can help to detect them. None of them is complete. Therefore we fetched data from five different sources. The following table lists these sources and the number of contained bots.

Note: The list is updateed manually thus disparities between this tables and the following results may occur! Last update: 28 May 2018

| criteria                           | amount       | source               | output file          |
| ---------------------------------- |:------------:|:--------------------:|:--------------------:|
| number of users in the group 'bot' | 245 | MediaWiki API (Example URL: https://www.wikidata.org/w/api.php?action=query&list=allusers&augroup=bot) | data/parser/users_in_bot_group.csv |
| number of bots that have a closed request for permission | 364 | [Requests for bot flags](https://www.wikidata.org/wiki/Wikidata:Requests_for_permissions/Archive#Requests_for_bot_flags ) | data/spiders/bots_with_requests_for_permissions.csv |
| number of bots with a botflag     | 258 | [Bots with botflag](https://www.wikidata.org/wiki/Category:Bots_with_botflag) | data/spiders/bots_with_botflag.csv |
| number of bots without a botflag  | 163 | [Bots without a botflag](https://www.wikidata.org/wiki/Category:Bots_without_botflag) | data/spiders/bots_without_botflag.csv |
| number of extension bots           | 7 | [Extension bots](https://www.wikidata.org/wiki/Category:Extension_bots) | data/spiders/extension_bots.csv |

### Number of bots with respect to there affiliation

The following statistics describe partially the same as the table before but instead of fetching the results from the csv files, they are retrived from the db. The calcuated values are summarizes in the subsequent matrix to give a better overview. An example of how to interpret the matrix is that the field `b_f b_f` describe how many distinct bots with a bot flag exist. If you want to see the names of the bots which belong to each group, you can uncomment the line ``` Analyser.print_matrix_bot_names() ```. The collumn names are described as follows:

| column name               | explanation                               |
| --------------------------|:-----------------------------------------:|
| b_f                       | has bot flag                              |
| no_b_f                    | has no bot flag                           |
| ex_b                      | is a extension bot                        |
| group                     | is in group 'bot'                         |
| request                   | has a request for permission              |
| s_request                 | has a successful request for permission   |
| u_request                 | has a unsuccessful request for permission |


In [22]:
Analyser.generate_matrix()

           b_f no_b_f ex_b group request s_request u_request
b_f        259      0    0   202     249       249        20
no_b_f     NaN    164    0    23      63        41        24
ex_b       NaN    NaN    6     3       1         1         0
group      NaN    NaN  NaN   252     242       242        15
request    NaN    NaN  NaN   NaN     368       322        70
s_request  NaN    NaN  NaN   NaN     NaN       322        24
u_request  NaN    NaN  NaN   NaN     NaN       NaN        70


In [21]:
# Analyser.print_matrix_bot_names()

### Count and names of unique bots

The following describes the number and the names of all unique bots. These bots are retrieved from all lists (e.g. bots with a bot flag, bots without a bot flag, bots with a request for permission, bots in the user group 'bot' and extension bots)

In [6]:
# Analyser.print_unique_bots()

### Differences between unique bots (with request vs. without request)
The following section compares all unique bots which have a request for permission and those who don't.
#### Count and names of bots with a request for permission.

In [36]:
# Analyser.print_bots_with_request()

#### Count and names of bots without a request for permission.

In [16]:
# Analyser.print_bots_without_request()

#### Rights of bots with a request for permission.
The following shows how many bots with a request for permission have a specific right. Note: the rights could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 364 and not 368!

In [23]:
# Analyser.print_rights_of_bots_with_request()

#### Bots with request and without rights in db

In [19]:
Analyser.print_bots_with_request_without_rights()

#################### Number of all bots with a request, without rights and a red link: ####################
 5 

#################### Names of all bots with a request, without rights and a red link: ####################
 AviBot, Epochs bot, Fabot, RavenXBot2, Shankarbot 

#################### Number of all bots with a request, without rights and without a red link: ####################
 1 

#################### Names of all bots with a request, without rights and without a red link: ####################
 Global Economic Map Bot 



All bots for which no rights could be retrived and which have a red link, were probably deleted. 

The bot 'Global Economic Map Bot' has its own user page but still no rights can be retrived over the api. It is not yet solved why (TODO!).

Its also not solved yet why there are 6 bots with no rights in the db, but the table above suggests that there are only 4 bots with no rights in the db (TODO!).

#### Rights of bots without a request for permission.
The following shows which rights bots without a request for permission have and how many of them have these right. Note: the rights could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 120 and not 122!

In [1]:
# Analyser.print_rights_of_bots_without_request()

#### Bots without request and without rights in db

In [7]:
# Analyser.print_bots_without_request_without_rights()

Der Bot 127.0.0.1 ist kein User. Er gehört zu den extension Bots.

Bots zu denen über die MediaWikiApi rechte abgefragt werden können aber nicht über die Wikidata:
* Eflybot

TODO: klären wie diese Exceptions gehandelt werden sollen

#### Differences of bot rights
The following shows all right that bots with a request for permission have but all other bots do not have and vice versa.

In [14]:
# Analyser.print_right_differences_for_request()

#### Groups of bots with a request for permission.
The following shows how many bots with a request for permission belong to a specific group. Note: the groups could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 349 and not 367!

Note: All visitors to the site, including unregistered users, are part of the '*' group and get associated rights (for more infromation see: https://en.wikipedia.org/wiki/Wikipedia:User_access_levels#User_groups)

In [108]:
# Analyser.print_groups_of_bots_with_request()

#### Bots with request and without groups in db

In [106]:
# Analyser.print_bots_with_request_without_groups()

Bots bei denen manuell über die API Gruppen abfragbar sind, aber keine Gruppen in der DB stehen (es ist noch nicht klar warum, aber die Vermutung ist, dass es bei den ersten vier an der Kodierung liegt):
* %C3%96dokBot
* D%C3%A6ghrefnBot
* %D4%B1%D5%B7%D5%A2%D5%B8%D5%BF
* %D0%9D%D0%A1%D0%91%D0%BE%D1%82
* Innocent bot: Der Name wurde manuell nachbearbeitet (vondaher wurden dazu keine Gruppen abgefragt)

Bots bei denen auch manuell über die API keine Gruppen abgefragt werden können:
* Global Economic Map Bot: Nicht klar warum
* Luuvabot: ist eigentlich A-lú-mih-bot (man wird redirected und nur unter A-lú-mih-bot sind Gruppen zu finden)
* DangSunFlood: ist eigentlich DangSunBot (man wird redirected und nur unter DangSunBot sind Gruppen zu finden)
* The Anomebot: ist eigentlich The Anomebot 3 (der Bot The Anomebot existiert nicht. Die Zahl wird fälschlicherweise weggeparsed)
* Theo%27s Little Bot: ist eigentlich Theo's Little Bot (Hier ist etwas mit der Kodierung nicht in Ordnung)
* DynamicBot Srv2: ist eigentlich DynBot Srv2 (man wird redirected und nur unter DynBot Srv2 sind Gruppen zu finden)
* Dima st bk bot: ist eigentlich DimaBot (man wird redirected und nur unter DimaBot sind Gruppen zu finden)
* DangSunFlood2: ist eigntlich DangSunBot2 (man wird redirected und nur unter DangSunBot2 sind Gruppen zu finden)

TODO: klären wie diese Exceptions gehandelt werden sollen

#### Groups of bots without a request for permission.
The following shows how many bots without a request for permission belong to a specific group. Note: the groups could not be retrived for all bots form the API for the reason that some bots for example do not have a user account anymore. Therefore in the graphic the maximum value of bots that have a specific right is 120 and not 122!

In [31]:
# Analyser.print_groups_of_bots_without_request()

#### Bots with request and without groups in db

In [29]:
# Analyser.print_bots_without_request_without_groups()

Der Bot 127.0.0.1 ist kein User. Er gehört zu den extension Bots.

Bots zu denen über die MediaWikiApi rechte abgefragt werden können aber nicht über die Wikidata:
* Eflybot

TODO: klären wie diese Exceptions gehandelt werden sollen

#### Differences of bot groups
The following shows all groups that bots with a request for permission belong to but all other bots do not and vice versa.

In [27]:
# Analyser.print_groups_differences()

### Differences between unique bots (with bot flag vs. bot in group bot)
The following section compares all unique bots which have a bot flag (and do not belong to the group bot) and bots which belong to the group bot (and do not have a bot flag).
#### Count and names of bots with a bot flag and do not belong to the group bot

In [53]:
# Analyser.print_bots_with_bot_flag()

#### Count and names of bots which belong to the group bot and do not have a bot flag

In [56]:
# Analyser.print_bots_in_bot_group()

#### Count and names of bots with a bot flag and which belong to the group bot
Note: if you add up this amount and the amount of 'Count and names of bots which belong to the group bot and do not have a bot flag' you will not get the overall amount of bot which belong to the bot group. This effect is due to the fact that in the db bots exist which belong to the bot group but have a NULL value for the field `has_botflag` and therefore are not captured by one of these queries.

In [58]:
# Analyser.print_bots_with_bot_flag_and_in_bot_group()

#### Rights of bots with a bot flag

In [42]:
# Analyser.print_rights_of_bot_with_bot_flag()

#### Rights of bots which belong to the group 'bot'

In [46]:
# Analyser.print_right_differences_for_bot_flag_and_bot_group()

#### Differences of bot rights
The following shows all right that bots with a bot flag have but bots which belong to the group bot do not have and vice versa. 

In [50]:
# Analyser.print_right_differences_for_bot_flag_and_bot_group()

### Analysis of Requests for Permission

This chepter aims to analyse the requests for permissions for bots on Wikidata.

| criteria                  | amount                |
| --------------------------|:---------------------:|
| number of bots            |356                    |
| number of requests        |655                    |
| date of the first request |2012-11-13 03:29:25    |
| date of the last request  |2018-05-14 13:03:51    |

As reference point the date when the request was first edited is taken.

#### Distribution of request for permission

This shows the distribution of request for permission over time. As reference point for the time the date when the request for permission was closed is taken.

In [110]:
Analyser.plot_distribution_over_time(
    '''
    SELECT closed_at 
    FROM requests_for_permissions 
    WHERE closed_at <> ''
    AND closed_at NOT NULL
    ORDER BY closed_at
    ''')

#### Distribution of successful request for permission

This shows the distribution of successful request for permission over time. As reference point for the time the date when the request for permission was closed is taken.

In [109]:
Analyser.plot_distribution_over_time(
    '''
    SELECT closed_at 
    FROM requests_for_permissions 
    WHERE is_successful = 1 
    AND closed_at <> '' 
    AND closed_at NOT NULL
    ORDER BY closed_at
    ''')

#### Distribution of unsuccessful request for permission

This shows the distribution of unsuccessful request for permission over time. As reference point for the time the date when the request for permission was closed is taken.

In [111]:
Analyser.plot_distribution_over_time(
    '''
    SELECT closed_at 
    FROM requests_for_permissions 
    WHERE is_successful = 0 
    AND closed_at <> ''
    AND closed_at NOT NULL
    ORDER BY closed_at
    ''')

#### Requests for permissions where closed_at is not set

TODO: klären, warum das so ist.

In [123]:
# Analyser.print_request_for_permission_without_closed_at()

#### Distribution of editor count

This shows the distribution of editor count per request for permission. The editor count represents the number of different users who edited the request for permission.

In [99]:
Analyser.plot_distribution(
    '''
    SELECT editor_count, COUNT(*)
    FROM requests_for_permissions
    WHERE editor_count NOT NULL
    GROUP BY editor_count
    ''')

#### Distribution of editor count with a successful request for permission

In [98]:
Analyser.plot_distribution(
    '''
    SELECT editor_count, COUNT(*)
    FROM requests_for_permissions
    WHERE editor_count NOT NULL
    AND is_successful = 1
    GROUP BY editor_count
    ''')

In [97]:
Analyser.plot_distribution(
    '''
    SELECT editor_count, COUNT(*)
    FROM requests_for_permissions
    WHERE editor_count NOT NULL
    AND is_successful = 0
    GROUP BY editor_count
    ''')

#### Requests for permissions where editor_count is not set

TODO: klären, warum das so ist.

In [122]:
# Analyser.print_request_for_permission_without_editor_count()